# Structuring and Summarizing Rechtspraak documents

 **Table of content:**
 - [Loading the data](#data-extraction)
 - [Statistics 'inhoudsindicatie'](#statistics-inhoud)
 - [Document structuring using headings clustering ](#head-cluster)
 - [Section summarization using XML data](#sec-sum)
 - [Full-text summarization using rechtspraak extractor data](#full-sum)
 - [Segmented summarization using XML data](#part-sum)
 - [Interesting Sources](#sources)

### Imports

In [ ]:
%pip install rechtspraak_extractor

In [1]:
# imports of generally used libraries
import os
import re
import string
import pandas as pd
import rechtspraak_extractor as rex
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

<a id="data-extraction"></a>
## Loading the data

In [2]:
rs_df = pd.read_csv("2022_rs_data.csv")

In [3]:
rs_df.head

<bound method NDFrame.head of                         ecli        date  \
0         ECLI:NL:CBB:2022:1  2022-01-11   
1        ECLI:NL:CBB:2022:10  2022-01-18   
2       ECLI:NL:CBB:2022:100  2022-03-08   
3       ECLI:NL:CBB:2022:101  2022-03-08   
4       ECLI:NL:CBB:2022:102  2022-03-08   
...                      ...         ...   
130330     ECLI:NL:XX:2022:5  2022-01-28   
130331     ECLI:NL:XX:2022:6  2022-02-28   
130332     ECLI:NL:XX:2022:7  2022-02-15   
130333     ECLI:NL:XX:2022:8  2022-03-28   
130334     ECLI:NL:XX:2022:9  2022-03-25   

                                         inhoudsindicatie  \
0       \n\nArtikel 2:3, eerste lid, van de Algemene w...   
1       \n\nMond - en klauwzeer. Medio maart 2001 is i...   
2       \n\n-\tBeleidsregel tegemoetkoming ondernemers...   
3       \nWarenwet, hoger beroep, Verordening 853/2004...   
4       \nRegeling garanties van oorsprong en certific...   
...                                                   ...   
130330        

<a id="statistics-inhoud"></a>
## Statistics 'inhoudsindicatie'

In this section, I perform some computations to get average length, shortest, longest and the distribution of length from all text under 'inhoudsindicatie' from 2022 cases

In [51]:
def get_inhoud(filepath):
    file = open(filepath, "r", encoding='utf-8')
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')
    
    try:
        inhoud = soup.find("inhoudsindicatie").text
    except:
        inhoud = ""
    
    inhoud = inhoud.replace("\n", "")
    #print(inhoud)
    
    return len(inhoud)
        

In [ ]:
path = 'unzip_data\\2022'

inhoud = []
counter = 0
for root, dirs, files in os.walk(path):
    print(len(files))
    for name in files:
        if counter % 10000 == 0:
            print(counter)
        #print(name)
        file = os.path.join(root, name)
        inhoud_length = get_inhoud(file)
        inhoud.append(inhoud_length)
        counter += 1

print(inhoud)

In [53]:
inhoud_copy = inhoud.copy()

uni, cnt = np.unique(inhoud_copy, return_counts=True)

print(uni)
print(cnt)

[    0     1     2 ... 11742 12492 12709]
[78903    78   297 ...     1     1     1]


In [57]:
np.mean(inhoud_copy)

130.6026853876549

In [59]:
np.average(inhoud_copy)

130.6026853876549

In [61]:
inhoud_no_zeros = [i for i in inhoud_copy if i != 0]
max(set(inhoud_no_zeros), key = inhoud_no_zeros.count)

49

In [65]:
out = uni[np.argsort(-cnt)]
print(out[:1000])

[   0   49    3   53   12   52    2   71  254    5  253  255   40  252
  197    6  196  193  194   56  250  120    7   79   50   76   31   35
   32  166  215  249  251   98   77  247   41   78  246   39  214   51
   64   57  216   94  121  198  242  118   45   33   69   75   70  243
   90   42   48  199  192   44   84   83   97   68   73  177   74  127
  245  134  116  248  124   61  175  165  122  203  244   37   63   54
  178   67   62   36   87  144  195   86  151  202   46  155   80   91
   81  117  102  133  179  174   85  108   66   93  104   89  105   60
   30   88  239  205  114  240  148  107   92  241  211   58   65   82
  101   43  110   55   95  212   25   47  236  146  137  159  112   99
  125  171  109  143  172  149  200   72  160  106  100  142   96  201
  147  111   34  164  119  237  136  129  217  154  173   59  131  130
  176  222   38  169    8  113  210  132  180  221  187  158  145  140
  139  126  235  152  170  181  115  138  156  103  226  123  191  213
  150 

<a id="head-cluster"></a>
## Document structuring using headings clustering 

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import advertools as adv

In [22]:
stopwords_dutch = list(adv.stopwords['dutch'])
print(stopwords_dutch)

['wil', 'je', 'in', 'dus', 'me', 'ook', 'heeft', 'het', 'aan', 'hoe', 'ja', 'kon', 'uit', 'zelf', 'met', 'wat', 'eens', 'door', 'hem', 'mij', 'ze', 'zo', 'zich', 'ge', 'hij', 'andere', 'haar', 'als', 'is', 'reeds', 'voor', 'waren', 'zei', 'geweest', 'men', 'niets', 'al', 'wij', 'en', 'niet', 'ben', 'heb', 'wel', 'uw', 'of', 'u', 'hebben', 'zijn', 'tot', 'hun', 'kunnen', 'meer', 'toch', 'er', 'we', 'veel', 'dat', 'de', 'nu', 'dit', 'om', 'tegen', 'ik', 'had', 'moet', 'zij', 'worden', 'naar', 'zonder', 'doen', 'over', 'kan', 'want', 'alles', 'zal', 'maar', 'af', 'altijd', 'hier', 'deze', 'iets', 'werd', 'wie', 'te', 'geen', 'bij', 'was', 'wezen', 'zou', 'toen', 'van', 'een', 'ons', 'der', 'dan', 'doch', 'daar', 'nog', 'mijn', 'op', 'die', 'iemand', 'omdat', 'na']


In [23]:
rs_metadata = rs_metadata.applymap(str)
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_dutch)
tfidf_matrix = tfidf_vectorizer.fit_transform(rs_metadata['headings'])

In [24]:
num_clusters = 12  # You can choose the number of clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)

KMeans(n_clusters=12)

In [25]:
cluster_labels = kmeans.labels_
rs_metadata = rs_metadata.assign(cluster_label=cluster_labels)

In [ ]:
print(rs_metadata)

In [27]:
rs_metadata.to_csv("cluster_labels.csv", index=False)

<a id="sec-sum"></a>
## Section summarization using XML data

<a id="full-sum"></a>
## Full-text summarization using rechtspraak extractor data

In this section, we will summarize the judgements based on the full-text extracted using the rechtspraak extractor package.

 **Table of content:**
 - [Clustering and Word2Vec](#clus-word2vec)
 - [Transformer models](#transformer)

<a id="clus-word2vec"></a>
### Clustering and Word2Vec

The steps we undertake are as follows:

- Split full-text into sentences
- Clean the data
- Get the vector representation of sentences
- Clustering
- Get summarized text

#### Loading the data with Rechtspraak extractor

In [10]:
df = rex.get_rechtspraak(max_ecli=1000, sd='2022-03-01', ed='2022-05-01', save_file='n')

df_metadata = rex.get_rechtspraak_metadata(save_file='n', dataframe=df)

INFO:root:Rechtspraak dump downloader API
INFO:root:Checking the API
INFO:root:API is working fine!
INFO:root:Getting 1000 documents from 2022-03-01 till 2022-05-01
INFO:root:Found 1000 cases!
INFO:root:Total execution time: 0:0:1.02
INFO:root:

INFO:root:Rechtspraak metadata API
INFO:root:Maximum 15 threads supported by your machine.
INFO:root:Getting metadata of 1000 ECLIs
INFO:root:Working. Please wait...
2999it [00:25, 31.21it/s]               ?, ?it/s]INFO:root:Total execution time: 0:0:27.39
INFO:root:



In [11]:
df_metadata

,ecli,full_text,creator,date_decision,issued,zaaknummer,type,relations,references,subject,procedure,inhoudsindicatie,hasVersion,summary
0,ECLI:NL:RBNHO:2022:1639,"\n\nRECHTBANK NOORD-HOLLAND\n\n\nTeam Straf, l...",Rechtbank Noord-Holland,2022-03-01,2022-03-01,15.116360.21,Uitspraak,,,Strafrecht,Eerste aanleg - meervoudig,\n\nVrijspraak deelname aan de voortzetting va...,\n\nRechtspraak.nl\n\n,Vrijspraak deelname aan de voortzetting van de...
1,ECLI:NL:CBB:2022:87,\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEG...,College van Beroep voor het bedrijfsleven,2022-03-01,2022-02-28,20/608,Uitspraak,,Meststoffenwet\n,Bestuursrecht,Eerste aanleg - enkelvoudig,"\nArtikel 8:88, eerste lid, aanhef en onder a,...",\n\nRechtspraak.nl\n\n,"Artikel 8:88, eerste lid, aanhef en onder a, i..."
2,ECLI:NL:RBMNE:2022:764,\n\nRECHTBANK MIDDEN-NEDERLAND\n\n\nStrafrecht...,Rechtbank Midden-Nederland,2022-03-01,2022-03-01,16/259657-19 (P),Uitspraak,,,Strafrecht,Eerste aanleg - meervoudig,\nVerdachte werkzaam als beveiliger. Vrijspraa...,\n\nRechtspraak.nl\n\n,Verdachte werkzaam als beveiliger. Vrijspraak ...
3,ECLI:NL:RBMNE:2022:762,\n\nRECHTBANK MIDDEN-NEDERLAND\n\n\nStrafrecht...,Rechtbank Midden-Nederland,2022-03-01,2022-03-01,16/098731-19 (TUL BIJZ),Uitspraak,,,Strafrecht,Eerste aanleg - meervoudig,\nToewijzing van de vordering tot tenuitvoerle...,\n\nRechtspraak.nl\n\n,Toewijzing van de vordering tot tenuitvoerlegg...
4,ECLI:NL:RBMNE:2022:759,\n\nRECHTBANK MIDDEN-NEDERLAND\n\n\nStrafrecht...,Rechtbank Midden-Nederland,2022-03-01,2022-03-01,16/211054-19 (P),Uitspraak,,,Strafrecht,Eerste aanleg - meervoudig,\nJeugdstrafrecht. Vrijspraak medeplichtigheid...,\n\nRechtspraak.nl\n\n,Jeugdstrafrecht. Vrijspraak medeplichtigheid b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ECLI:NL:RBDHA:2022:1844,\n\nRECHTBANK DEN HAAG\n\n\nZittingsplaats Mid...,Rechtbank Den Haag,2022-03-02,2022-03-08,NL21.19965,Uitspraak,,,Bestuursrecht; Vreemdelingenrecht,Eerste aanleg - enkelvoudig,"\nafgewezen asielaanvraag, magazine, mensenhan...",\n\nRechtspraak.nl\n\n,"afgewezen asielaanvraag, magazine, mensenhande..."
996,ECLI:NL:RBGEL:2022:1227,\n\n\n\n\nRECHTBANK GELDERLAND\n\n\n\n\nTeam s...,Rechtbank Gelderland,2022-03-08,2022-03-08,05/281861-21,Uitspraak,,,Strafrecht,Eerste aanleg - meervoudig,\ntaakstraf van 120 uren en een voorwaardelijk...,\n\nRechtspraak.nl\n\n,taakstraf van 120 uren en een voorwaardelijke ...
997,ECLI:NL:RBDHA:2022:1845,\n\n\nRECHTBANK DEN HAAG\n\n\nZittingsplaats M...,Rechtbank Den Haag,2022-03-02,2022-03-08,NL22.2349,Uitspraak,,,Bestuursrecht; Vreemdelingenrecht,Eerste aanleg - enkelvoudig,\nDe trefwoorden van bijgevoegde uitspraak: Be...,\n\nRechtspraak.nl\n\n,De trefwoorden van bijgevoegde uitspraak: Bewa...
998,ECLI:NL:GHARL:2022:1698,\nAfdeling strafrecht\n\nParketnummer:\t\t21-0...,Gerechtshof Arnhem-Leeuwarden,2022-03-08,2022-03-07,21-001102-21,Uitspraak,,,Strafrecht,Hoger beroep,\nHet hof verklaart bewezen dat de verdachte z...,\n\nRechtspraak.nl\n\n,Het hof verklaart bewezen dat de verdachte zic...


In [25]:
filtered_df = df_metadata[df_metadata['ecli'].str.contains('ECLI:NL:CBB:')]
filtered_df

,ecli,full_text,creator,date_decision,issued,zaaknummer,type,relations,references,subject,procedure,inhoudsindicatie,hasVersion,summary
1,ECLI:NL:CBB:2022:87,\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEG...,College van Beroep voor het bedrijfsleven,2022-03-01,2022-02-28,20/608,Uitspraak,,Meststoffenwet\n,Bestuursrecht,Eerste aanleg - enkelvoudig,"\nArtikel 8:88, eerste lid, aanhef en onder a,...",\n\nRechtspraak.nl\n\n,"Artikel 8:88, eerste lid, aanhef en onder a, i..."
11,ECLI:NL:CBB:2022:91,\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEG...,College van Beroep voor het bedrijfsleven,2022-03-01,2022-02-28,19/1672,Uitspraak,,Wet tuchtrechtspraak accountants\n,Bestuursrecht,Hoger beroep,\n\nAccountantstuchtrecht. \nDe toetsers van d...,\n\nRechtspraak.nl\n\n,Accountantstuchtrecht. \nDe toetsers van de NB...
26,ECLI:NL:CBB:2022:86,,College van Beroep voor het bedrijfsleven,2022-03-01,2022-02-28,19/1995,Uitspraak,,Meststoffenwet\n,Bestuursrecht,Eerste aanleg - enkelvoudig,,,No summary available
951,ECLI:NL:CBB:2022:103,\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,20/429,Uitspraak,,Kaderwet EZK- en LNV-subsidies\n,Bestuursrecht,Eerste aanleg - meervoudig,\nRegeling garanties van oorsprong en certific...,\n\nRechtspraak.nl\n\n,Regeling garanties van oorsprong en certificat...
959,ECLI:NL:CBB:2022:102,\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,"20/430 tm 20/433, 21/476 tm 21/478",Uitspraak,,Kaderwet EZK- en LNV-subsidies\n,Bestuursrecht,Eerste aanleg - meervoudig,\nRegeling garanties van oorsprong en certific...,\n\nRechtspraak.nl\n\n,Regeling garanties van oorsprong en certificat...
970,ECLI:NL:CBB:2022:100,\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-04,21/653,Uitspraak,,,Bestuursrecht,Eerste aanleg - enkelvoudig,\n\n-\tBeleidsregel tegemoetkoming ondernemers...,\n\nRechtspraak.nl\n\n,-\tBeleidsregel tegemoetkoming ondernemers get...
972,ECLI:NL:CBB:2022:104,\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,20/820,Uitspraak,,Kaderwet EZK- en LNV-subsidies\n,Bestuursrecht,Eerste aanleg - meervoudig,\nIntrekking SDE-subsidie zonnepanelen. Artike...,\n\nRechtspraak.nl\n\n,Intrekking SDE-subsidie zonnepanelen. Artikele...
973,ECLI:NL:CBB:2022:105,\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEG...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,21/835,Uitspraak,,Wet tuchtrechtspraak accountants\n,Bestuursrecht,Hoger beroep,\naccountantstucht; hoger beroep niet-ontvanke...,\n\nRechtspraak.nl\n\n,accountantstucht; hoger beroep niet-ontvankeli...
974,ECLI:NL:CBB:2022:107,\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,20/1100,Uitspraak,,Meststoffenwet\n,Bestuursrecht,Eerste aanleg - enkelvoudig,\nArtikel 8:88 van de Algemene wet bestuursrec...,\n\nRechtspraak.nl\n\n,Artikel 8:88 van de Algemene wet bestuursrecht...
975,ECLI:NL:CBB:2022:108,\n\nuitspraak\n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE...,College van Beroep voor het bedrijfsleven,2022-03-08,2022-03-07,20/829,Uitspraak,,Gaswet\n,Bestuursrecht,Eerste aanleg - meervoudig,\nTarievenbesluit Gas 2020. Nieuwe taak van in...,\n\nRechtspraak.nl\n\n,Tarievenbesluit Gas 2020. Nieuwe taak van inko...


In [69]:
text = df_metadata.iloc[1].full_text
print(repr(text))

'\n\nuitspraak \n\n\n\n\n\n\n\n\n\n\n\n\nCOLLEGE VAN BEROEP VOOR HET BEDRIJFSLEVEN\n\n\nzaaknummer: 20/608 \n\n\nuitspraak van de enkelvoudige kamer van 1 maart 2022 in de zaak tussen\nMaatschap [naam 1] en [naam 2] , te [plaats] , appellante\n(gemachtigde: J.A. Rietveld),\n\n\nen\n\n\nde minister van Landbouw, Natuur en Voedselkwaliteit, verweerder\n(gemachtigde: mr. A. Herczog),\n\n\nen\n\n\nde Staat der Nederlanden (de minister van Justitie en Veiligheid), de Staat.\n\n\n\nProcesverloop \n\n\nBij besluit van 13 januari 2018 (het primaire besluit) heeft verweerder op grond van artikel 23, derde lid, van de Meststoffenwet (Msw) het fosfaatrecht van appellante vastgesteld. \n\n\n\nBij besluit van 25 september 2018 (het herzieningsbesluit) heeft verweerder het primaire besluit herroepen en het fosfaatrecht van appellante lager vastgesteld. \n\n\n\nBij besluit van 23 oktober 2019 (het vervangingsbesluit) heeft verweerder het herzieningsbesluit herroepen en het fosfaatrecht van appellante

In [75]:
text = df_metadata.iloc[1].full_text
#text = text.replace('\n', ' ')
#text = text.replace('\n\n ', '\n')
#text = text.replace('\n\n\n', '\n\n')
#text = text.replace('\n\n\n\n', '\n\n\n')
split = text.split('\n\n')
for sent in split:
    print(repr(sent))
#print(split)
#print(df_metadata.iloc[1].ecli)
#print(text)

''
'uitspraak '
''
''
''
''
''
'\nCOLLEGE VAN BEROEP VOOR HET BEDRIJFSLEVEN'
'\nzaaknummer: 20/608 '
'\nuitspraak van de enkelvoudige kamer van 1 maart 2022 in de zaak tussen\nMaatschap [naam 1] en [naam 2] , te [plaats] , appellante\n(gemachtigde: J.A. Rietveld),'
'\nen'
'\nde minister van Landbouw, Natuur en Voedselkwaliteit, verweerder\n(gemachtigde: mr. A. Herczog),'
'\nen'
'\nde Staat der Nederlanden (de minister van Justitie en Veiligheid), de Staat.'
''
'Procesverloop '
'\nBij besluit van 13 januari 2018 (het primaire besluit) heeft verweerder op grond van artikel 23, derde lid, van de Meststoffenwet (Msw) het fosfaatrecht van appellante vastgesteld. '
''
'Bij besluit van 25 september 2018 (het herzieningsbesluit) heeft verweerder het primaire besluit herroepen en het fosfaatrecht van appellante lager vastgesteld. '
''
'Bij besluit van 23 oktober 2019 (het vervangingsbesluit) heeft verweerder het herzieningsbesluit herroepen en het fosfaatrecht van appellante hoger vastgesteld. 

2000it [4:17:19, 42.16it/s]000/1000 [4:19:53<00:00, 33.08it/s]

In [30]:
text1 = df_metadata.iloc[977].full_text
print(df_metadata.iloc[977].ecli)
print(text1)

ECLI:NL:CBB:2022:98


uitspraak












COLLEGE VAN BEROEP VOOR HET BEDRIJFSLEVEN


zaaknummer: 21/616 


uitspraak van de enkelvoudige kamer van 8 maart 2022 in de zaak tussen

      [naam 1] B.V., te [woonplaats 1] , appellante
(gemachtigde: J.J. Tabak),


en


de minister van Economische Zaken en Klimaat, verweerder 
(gemachtigden: mr. M.J.H. van der Burgt en mr. S.M. Piron).



Procesverloop 


Bij besluit van 23 september 2020 (het primaire besluit) heeft verweerder aan appellante op grond van de Regeling subsidie financiering vaste lasten MKB COVID-19 (TVL) een voorlopige subsidie toegekend van € 5.026,82. 



Bij besluit van 30 april 2021 (het bestreden besluit) heeft verweerder het bezwaar van appellante ongegrond verklaard. 



Appellante heeft tegen het bestreden besluit beroep ingesteld. 



Verweerder heeft een verweerschrift ingediend.



Het onderzoek ter zitting heeft plaatsgevonden op 31 januari 2022. Namens appellante zijn verschenen [naam 2] en [naam 3] , bijgesta

In [39]:
spli = text.split('Procesverloop')
spli2 = spli[1].split('Overwegingen')
print(spli2)

[' \n\n\nBij besluit van 13 januari 2018 (het primaire besluit) heeft verweerder op grond van artikel 23, derde lid, van de Meststoffenwet (Msw) het fosfaatrecht van appellante vastgesteld. \n\n\n\nBij besluit van 25 september 2018 (het herzieningsbesluit) heeft verweerder het primaire besluit herroepen en het fosfaatrecht van appellante lager vastgesteld. \n\n\n\nBij besluit van 23 oktober 2019 (het vervangingsbesluit) heeft verweerder het herzieningsbesluit herroepen en het fosfaatrecht van appellante hoger vastgesteld. \n\n\n\nBij besluit van 1 mei 2020 (het bestreden besluit) heeft verweerder het vervangingsbesluit gehandhaafd. \n\n\n\nAppellante heeft beroep ingesteld tegen het bestreden besluit. Zij heeft daarbij tevens verzocht om schadevergoeding in verband met onrechtmatige besluitvorming en overschrijding van de redelijke termijn.\n\n\n\nVerweerder heeft een verweerschrift ingediend.\n\n\n\nHet onderzoek ter zitting heeft plaatsgevonden op 18 november 2021. Namens appellante 

In [ ]:
text = text.replace('\n', ' ')
text = re.sub(r'\s+', ' ', text)
print(text.strip())

#### Split the data into sentences

In [32]:
import nltk
nltk.download('punkt')   # one time execution
from nltk.tokenize import sent_tokenize
sentence = sent_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


#### Clean the text

In [33]:
import re
nltk.download('stopwords')  # one time execution
from nltk.corpus import stopwords
corpus = []
for i in range(len(sentence)):
    sen = re.sub('[^a-zA-Z]', " ", sentence[i])  
    sen = sen.lower()                            
    sen = sen.split()                         
    sen = ' '.join([i for i in sen if i not in stopwords.words('dutch')])   
    corpus.append(sen)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chloe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


#### Vector representation

In [35]:
from gensim.models import Word2Vec
all_words = [i.split() for i in corpus]
model = Word2Vec(all_words, min_count=1, vector_size= 300)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:collected 598 word types from a corpus of 1167 raw words and 87 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 598 unique words (100.0%% of original 598, drops 0)', 'datetime': '2023-11-20T16:01:04.416542', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 1167 word corpus (100.0%% of original 1167, drops 0)', 'datetime': '2023-11-20T16:01:04.417543', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'

In [36]:
sent_vector=[]
for i in corpus:
    
    plus=0
    for j in i.split():
        plus+= model.wv[j]
    plus = plus/len(i.split())
    
    sent_vector.append(plus)

#### Clustering

In [37]:
import numpy as np
from sklearn.cluster import KMeans
n_clusters = 5
kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(sent_vector)

In [38]:
from scipy.spatial import distance
my_list=[]
for i in range(n_clusters):
    my_dict={}
    
    for j in range(len(y_kmeans)):
        
        if y_kmeans[j]==i:
            my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))
 

#### The summary

In [39]:
for i in sorted(my_list):
    print(sentence[i])

Bij besluit van 27 oktober 1998 heeft verweerder met gebruikmaking van de op 22 juli 1998 van gedeputeerde staten van Gelderland ontvangen verklaring van geen bezwaar aan Grondexploitatiemaatschappij Waalsprong (GEM) (verder: vergunninghouder sub 2), vrijstelling verleend van het bestemmingsplan "Buitengebied Valburg" ten aanzien van bestemming, gebruik, bebouwing en aanleg van werken ten behoeve van het bouwrijpmaken voor de woningbouw eerste fase Woonpark Oosterhout (Oosterhout Midden).
II.
Overwegingen.
Beslissing.
van der Bend als griffier.


100%|██████████| 1000/1000 [3:07:57<00:00, 31.78it/s]

<a id="transformer"></a>
### Transformer models

#### BERT

In [ ]:
!pip install spacy

In [4]:
from summarizer import Summarizer, TransformerSummarizer

In [38]:
def clean_text(text):
    text = text.replace('\n',' ')
    text = re.sub(" +", " ", text)
    text = re.sub(r' (?<!\S)\d+(\.\d+)+(?!\S) ', '', text)
    
    dutch_headers = [
    "Inleiding", "Samenvatting", "Achtergrond", "Methodologie", "Resultaten",
    "Discussie", "Conclusie", "Literatuuronderzoek", "Onderzoeksvraag", "Doelstelling",
    "Materiaal en Methoden", "Analyse", "Bespreking van Resultaten", "Implicaties",
    "Toekomstig Onderzoek", "Referenties", "Bijlagen", "Verantwoording", "Abstract",
    "Probleemstelling", "Onderzoeksmethode", "Data-analyse", "Statistische Analyse",
    "Experimenteel Ontwerp", "Case Study", "Literatuuroverzicht", "Conceptueel Kader",
    "Hypothesen", "Onderzoeksopzet", "Onderzoekspopulatie", "Steekproefomvang",
    "Variabelen", "Meetinstrumenten", "Validiteit", "Betrouwbaarheid", "Resultaatinterpretatie",
    "Kritische Reflectie", "Praktische Implicaties", "Beperkingen van het Onderzoek",
    "Aanbevelingen", "Literatuurlijst", "Voetnoten", "Begrippenlijst", "Dankwoord",
    "Voorwoord", "Abstract", "Theoretisch Kader", "Ethische Overwegingen"
    ]
    dutch_head = '|'.join(dutch_headers)

    text = re.sub(dutch_head, '',text)
    text = text.strip()
    return text

In [45]:
bert_model = Summarizer()
idx = 16
procesverloop = clean_text(rs_df['procesverloop'][idx])
overwegingen = clean_text(rs_df['overwegingen'][idx])
beslissing =  clean_text(rs_df['beslissing'][idx])
proces_summary = ''.join(bert_model(procesverloop, min_length=10, max_length=150))
overw_summary = ''.join(bert_model(overwegingen, min_length=10, max_length=150))
beslis_summary = ''.join(bert_model(beslissing, min_length=10, max_length=150))
print("ecli: ", rs_df['ecli' ][idx])
print("inhoudsindicatie: ", rs_df['inhoudsindicatie'][idx])
print("BERT summary: ", proces_summary, '\n' , overw_summary,'\n' , beslis_summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Chloe\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when t

ecli:  ECLI:NL:CBB:2022:113
inhoudsindicatie:  
Wet marktordening gezondheidszorg, artikel 47 en 48

Marktanalysebesluit gericht aan een zorgaanbieder. Bij het marktanalysebesluit is aan de zorgaanbieder een transparantieverplichting en een contracteerverplichting opgelegd omdat de zorgaanbieder volgens de NZa op een aantal relevante markten over aanmerkelijke marktmacht (AMM) beschikt waardoor sprake is van (potentiële) mededingingsproblemen.

Het College komt tot het oordeel dat de zorgaanbieder over AMM beschikt. Er is geen sprake van een zodanige afnemersmacht bij de zorgverzekeraars dat de marktmacht van de zorgaanbieder daardoor wordt gecompenseerd. Er is sprake van (potentiële) mededingingsproblemen. Aangezien de opgelegde verplichtingen geschikt zijn om de door de NZa geconstateerde mededingingsproblemen (uitbuiting) op te lossen, de oplossing daarvan noodzakelijk is om zorgverzekeraars hun door de wetgever beoogde rol in het zorgstelsel waar te laten maken, gekozen is voor de 

#### GPT 2

In [7]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(text, min_length=20))
print(full)

Bij besluit van 2 oktober 2020 (het primaire besluit) heeft verweerder beslist op de aanvraag van appellant om een investeringssubsidie duurzame energie (ISDE) voor een warmtepomp in het kader van de Regeling nationale EZ-subsidies (Regeling). Verweerder heeft zich laten vertegenwoordigen door zijn gemachtigde. Bij brief van 20 oktober 2021 heeft het College verweerder verzocht zijn standpunt nader toe te lichten.


<a id="part-sum"></a>
## Segmented summarization using XML data

In [ ]:
# load csv

## Evaluating Summarization Methods

<a id="sources"></a>
### Interesting Sources

<b>Extractive Summarization - full text</b>
- [Summarization using k-means clustering](https://medium.com/@akankshagupta371/understanding-text-summarization-using-k-means-clustering-6487d5d37255)
- [Understanding Text Summarization](https://medium.com/towards-data-science/understanding-automatic-text-summarization-1-extractive-methods-8eb512b21ecc)
- [Extractive summarization using transformers](https://medium.com/analytics-vidhya/text-summarization-using-bert-gpt2-xlnet-5ee80608e961)

<b>Query-based Summarization</b>
- [Query-based Summarization explanation](https://medium.com/@fenil.h.dedhia/query-based-summarization-in-action-ea729df3109c)
- [Agolo's Query-based summarization](https://medium.com/@abdarhman.abdelhamid/agolos-query-focused-summarization-uses-information-retrieval-methods-for-open-domain-5b52eca09bac)
- [Query-based Summarization tutorial using Azure](https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/query-based-summarization)

